# Eksploracja Danych - Projekt
Tomasz Kiljańczyk (136257)

Wojciech Lulek (136280)

In [ ]:
import os
from multiprocessing import Pool

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm_notebook

In [ ]:
us_trending_df = pd.read_csv('./data/us_trending_stage_1.csv').head(1000)
us_trending_df

In [ ]:
us_trending_df['thumbnail_link_hires'] = us_trending_df['thumbnail_link'].str.replace('default.jpg', '0.jpg')
urls = us_trending_df['thumbnail_link_hires']
unique_urls = us_trending_df['thumbnail_link_hires'].unique()
us_trending_df['thumbnail_link_hires']

In [ ]:
from notebooks.workers import extract_color_features
from PIL import Image

results = []

with Pool(processes=os.cpu_count()) as pool:
    for res in tqdm_notebook(pool.imap(extract_color_features, unique_urls), total=unique_urls.size):
        results.append(res)

color_features_dict = dict(results)

color_features = [color_features_dict[url] for url in urls]

image_dataframe = pd.DataFrame(color_features)
image_dataframe

In [ ]:
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst


for i, url in enumerate(urls.head(5)):
    file_path = f'./images/{url.split("/")[-2]}.jpg'

    image = Image.open(file_path)

    row = image_dataframe.iloc[i]
    median_hsv_color = (
        round(row['thumbnail_med_hue']),
        round(row['thumbnail_med_saturation']),
        round(row['thumbnail_med_value'])
    )
    avg_hsv_color = (
        round(row['thumbnail_avg_hue']),
        round(row['thumbnail_avg_saturation']),
        round(row['thumbnail_avg_value'])
    )

    dims = (round(image.width * 0.1), image.height)
    median_color_image = Image.new('HSV', dims, color=median_hsv_color).convert('RGB')
    average_color_image = Image.new('HSV', dims, color=avg_hsv_color).convert('RGB')

    image_result_1 = get_concat_h(image, median_color_image)
    image_result_1 = get_concat_h(image_result_1, average_color_image)

    display(image_result_1)

In [ ]:
from notebooks.workers import detect_text

results = []
with Pool(processes=os.cpu_count()) as pool:
    for res in tqdm_notebook(pool.imap(detect_text, unique_urls), total=unique_urls.size):
        results.append(res)

text_features_dict = dict(results)

In [ ]:
has_text = [text_features_dict.get(url, (False,))[0] for url in urls]
text_count = [text_features_dict.get(url, (None, 0))[1] for url in urls]
largest_text_to_image_area_ratio = [text_features_dict.get(url, (None, 0, 0))[2] for url in urls]

In [ ]:
has_text_df = pd.DataFrame(has_text, columns=['thumbnail_has_text'])
has_text_df

In [ ]:
text_count_df = pd.DataFrame(text_count, columns=['thumbnail_text_count'])
text_count_df

In [ ]:
largest_text_to_image_area_ratio_df = pd.DataFrame(largest_text_to_image_area_ratio,
                                                   columns=['thumbnail_largest_text_to_image_area_ratio'])
largest_text_to_image_area_ratio_df

In [ ]:
final_df = pd.concat([image_dataframe, has_text_df], axis=1)
final_df = pd.concat([final_df, text_count_df], axis=1)
final_df = pd.concat([final_df, largest_text_to_image_area_ratio_df], axis=1)
final_df

In [ ]:
corrMatrix = final_df.corr(method='spearman')
corrMatrix = corrMatrix.round(4)

mask = np.triu(np.ones_like(corrMatrix, dtype=bool))
np.fill_diagonal(mask, False)

fig, ax = plt.subplots(figsize=(30, 30))
sns.heatmap(corrMatrix, mask=mask, annot=True, fmt='g', ax=ax)
plt.show()

In [ ]:
final_df.to_csv("./stage_2_thumbnail_colors_and_text.csv", index=False)